In [ ]:
%matplotlib qt5

In [ ]:
import matplotlib.pyplot as plt
from pddc_helpers import (load_bwi_data, aggregate_by_day, extract_day_of_hourly,
                          label_date)
import uuid
#plt.ion()

# Picking
We touched a bit on this in earlier examples, but let's now dive a bit deeper into "picking". Picking is built into Matplotlib's architecture. Automatically, for any 'button_press_event', the `contains()` method for all artist objects (anything that is drawable on the canvas is an artist) is called for the given x/y locations. Each artist may define what their `contains()` method means, but the important thing is that if `contains()` returns True, then a 'pick_event' is emitted with a specialized event object.

That event object has a reference to the `artist` object that passed the `contains()` test, along with other event information such as the x/y coordinates and such. If the artist is of a special type known as a Collection, then the event will also have a list of integer indexes, `ind`, that represents which components of the artist collection was near the mouse event.

In this example, we will create the callable class `RowPrinter` that will take a `LineCollection` object from `plt.plot()`, a pandas dataframe, and a picker argument. Previously, we set the 'picker' argument in the call to `plt.plot()`, but it doesn't have to be done that way. The `RowPrinter` will connect itself to pick events, and will print out information about any lines that are close to the mouse click.

In [ ]:
class RowPrinter:
    def __init__(self, ln, df, picker=10):
        ln.set_picker(picker)
        self.uid = str(uuid.uuid4())
        ln.set_gid(self.uid)
        self.ln = ln
        self.df = df
        self.cid = None
        self.connect()

    def connect(self):
        self.remove()
        self.cid = self.ln.figure.canvas.mpl_connect('pick_event',
                                                     self)

    def __call__(self, event):
        # ignore picks on not-our-artist
        if event.artist is not self.ln:
            return
        # for each hit index, print out the row
        for i in event.ind:
            print(self.df.iloc[i])

    def remove(self):
        if self.cid is not None:
            self.ln.figure.canvas.mpl_disconnect(self.cid)
            self.cid = None

In [ ]:
# Loading the dataframe object. Keep only the data after 2014.
bwi = load_bwi_data()
bwi = bwi[bwi['year'] > 2014]
bwi_daily = aggregate_by_day(bwi)

Display the daily average temperature average since 2014 at a given location.

The `RowPrinter` object is given the displayed line artist, as well as the daily temperatures as a time series. When we click on a spot, we find out what the specific values are for near that spot in the time series.

In [ ]:
fig, ax = plt.subplots()
fig.canvas.mpl_disconnect(fig.canvas.manager.key_press_handler_id)
ln, = ax.plot('mean', '-o', data=bwi_daily)
ax.set_xlabel('Date [UTC]')
ax.set_ylabel('Air Temperature [℃]')
ax.set_title('BWI')
rp = RowPrinter(ln, bwi_daily)

#one_day = extract_day_of_hourly(bwi, 2015, 10, 18)

# EXERCISE
- make the print out nicer looking
- make picking add a label with `label_data`
- use `get_gid` to filter instead of `is not`
- open a new window with plot of day temperature
  - fig, ax = plt.subplots()
  - one_day = extract_day_of_hourly(bwi, 2015, 10, 18)